In [1]:
# !apt update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
# !tar -xvf spark-3.3.0-bin-hadoop3.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
import findspark
findspark.init()

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [3]:
# %cd '/content/gdrive/My Drive/LDS9/Practice/Chapter6/'

In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

In [5]:
sc =SparkContext()

In [6]:
spark = SparkSession(sc)

## Chuẩn bị, chuẩn hóa dữ liệu, xác định input, output

In [7]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("flights.csv", inferSchema=True,header=True)

In [9]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [10]:
print((data.count(), len(data.columns)))

(50000, 10)


In [11]:
# Khi in bằng head thì định dạng hiển thị là row (khác với head ở pandas dataframe)
data.head(1)

[Row(mon=11, dom=20, dow=6, carrier='US', flight=19, org='JFK', mile=2153, depart=9.48, duration=351, delay='NA')]

In [12]:
data.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 3 rows



In [13]:
for item in data.head():
    print(item)

11
20
6
US
19
JFK
2153
9.48
351
NA


In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
data.columns

['mon',
 'dom',
 'dow',
 'carrier',
 'flight',
 'org',
 'mile',
 'depart',
 'duration',
 'delay']

In [16]:
assembler = VectorAssembler(
    inputCols=['mile'],
    outputCol='features'
)

In [17]:
data_pre = assembler.transform(data)

In [19]:
data_pre.select('features').show(2, False)

+--------+
|features|
+--------+
|[2153.0]|
|[316.0] |
+--------+
only showing top 2 rows



In [20]:
data_pre.show(2)

+---+---+---+-------+------+---+----+------+--------+-----+--------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|features|
+---+---+---+-------+------+---+----+------+--------+-----+--------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|[2153.0]|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| [316.0]|
+---+---+---+-------+------+---+----+------+--------+-----+--------+
only showing top 2 rows



In [21]:
final_data = data_pre.select('features','duration')

In [22]:
final_data.count()

50000

In [23]:
train_data, test_data = final_data.randomSplit([0.8,0.2])

In [24]:
train_data.describe().show()

+-------+-----------------+
|summary|         duration|
+-------+-----------------+
|  count|            40038|
|   mean|152.1450372146461|
| stddev|87.36586195376904|
|    min|               30|
|    max|              560|
+-------+-----------------+



In [24]:
train_data.describe().show()

+-------+-----------------+
|summary|         duration|
+-------+-----------------+
|  count|            40038|
|   mean|152.1450372146461|
| stddev|87.36586195376904|
|    min|               30|
|    max|              560|
+-------+-----------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                152|
|   mean| 506.70337866711077|
| stddev|  73.41168339420653|
|    min|  314.4385182951061|
|    max|  684.1634310159512|
+-------+-------------------+



In [42]:
lr = LinearRegression(featuresCol='features',
                     labelCol='duration',
                     predictionCol='prediction')

In [43]:
lrModel = lr.fit(train_data)

In [44]:
print('Coefficiets: {} Intercept: {}'.format(lrModel.coefficients, lrModel.intercept))

Coefficiets: [0.12159633940590948] Intercept: 44.497688181904444


In [45]:
test_result =lrModel.evaluate(test_data)

In [46]:
test_result.residuals.show(5)

+------------------+
|         residuals|
+------------------+
|-8.644642922100381|
|-6.644642922100381|
|-6.644642922100381|
|-6.644642922100381|
|-6.644642922100381|
+------------------+
only showing top 5 rows



In [47]:
print('RMSE:', test_result.rootMeanSquaredError)
print('MSE:', test_result.meanSquaredError)
print('r2:', test_result.r2)

RMSE: 16.918740463304054
MSE: 286.24377886464185
r2: 0.961050424235849


In [48]:
test_model = lrModel.transform(test_data)

In [49]:
test_model.select('prediction','duration').show(5)

+-----------------+--------+
|       prediction|duration|
+-----------------+--------+
|52.64464292210038|      44|
|52.64464292210038|      46|
|52.64464292210038|      46|
|52.64464292210038|      46|
|52.64464292210038|      46|
+-----------------+--------+
only showing top 5 rows



In [50]:
lrModel.save('lrModel_flights')

Py4JJavaError: An error occurred while calling o476.save.
: java.io.IOException: Path lrModel_flights already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractComman

In [51]:
from pyspark.ml.regression import LinearRegressionModel
lrModel2 = LinearRegressionModel.load('lrModel_flights')

In [35]:
unlabeled_data = test_data.select('features')

In [37]:
predictions = lrModel2.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+---------------------------+
|            features|Predict_Yearly Amount Spent|
+--------------------+---------------------------+
|[30.7377203726281...|         451.57104601510105|
|[30.9716756438877...|         488.33141581037376|
|[31.0662181616375...|         462.22982846298873|
|[31.3895854806643...|         409.55841803055637|
|[31.5702008293202...|          564.2584749230625|
|[31.5761319713222...|          543.7225862249902|
|[31.6610498227460...|          417.4199418533685|
|[31.7216523605090...|         349.47196093116463|
|[31.8209982016720...|         417.21897934964363|
|[31.8293464559211...|          384.1294209318296|
|[31.8530748017465...|         461.91362228654316|
|[31.8627411090001...|          558.7623419892514|
|[31.8648325480987...|          450.5290615648105|
|[31.8745516945853...|         398.05252963956286|
|[31.9096268275227...|          552.4008482087479|
|[31.9453957483445...|           663.540963795514|
|[31.9563005605233...|         

In [52]:
from pyspark.ml.evaluation import RegressionEvaluator

In [53]:
RegressionEvaluator(labelCol='duration').evaluate(test_model)

16.918740463304054